In [1]:
from MAP import *

In [10]:
def split_byteCode(byteCode,isDeploy=True):
    contract_start=0
    contract_end=0
    bzzr_start=0
    augs_start=0
    deployment=''
    contract=''
    bzzr=''
    augs=[]

    byteCode=byteCode.lower()
    if isDeploy:
        contract_start=byteCode.find('f300')
        contract_start+=4

    bzzr_start=byteCode.find('a165627a7a72305820')
    bzzr='bzzr: '+byteCode[bzzr_start+18:bzzr_start+18+64]
    augs_start=bzzr_start+18+64+4

    for i in range(augs_start,len(byteCode),64):
        aug=str('Arg [{}]: '.format(str((i-augs_start) // 64))+byteCode[i:i+64])
        augs.append(aug)
    deployment=byteCode[:contract_start]
    contract=byteCode[contract_start:bzzr_start]
    return deployment,contract,bzzr,augs

In [32]:
import os

with open('cro.bin','r') as f:
    binary=f.read()

In [33]:
deploy,cont,bzzr,aug=split_byteCode(binary)

In [34]:
from pprint import pprint
print(deploy)
print(cont)
print(bzzr)
pprint(aug)

60806040526004805460a060020a60ff02191690556006805460ff199081169091556009805490911660011790553480156200003a57600080fd5b5060405162002726380380620027268339810180604052810190808051820192919060200180518201929190602001805190602001909291908051906020019092919080519060200190929190805190602001909291908051906020019092919080519060200190929190805190602001909291908051906020019092919050505060008060003333600360006101000a815481600160a060020a030219169083600160a060020a0316021790555080600660016101000a815481600160a060020a030219169083600160a060020a031602179055505033600360006101000a815481600160a060020a030219169083600160a060020a03160217905550600360009054906101000a9004600160a060020a0316600460006101000a815481600160a060020a030219169083600160a060020a031602179055508c600a90805190602001906200018c929190620006a4565b508b51620001a290600b9060208f0190620006a4565b50600c805460ff191660ff8c161761010060a860020a031916610100600160a060020a038b81169190910291909117909155600d8054600160a060020a03199081168a841617909155600e805482168984

In [39]:
def get_opCode_list(byteCode):
    opCode_list=[]
    push_skip_times=0
    for i in range(0,len(byteCode),2):
        code=byteCode[i:i+2]
        if push_skip_times==0:
            loc=str(hex(i)).zfill(10)
            if code[0] != '6' and code[0] != '7':
                opCode=[loc,code,opcodes.get(code,'<------- ERROR'),'']
                opCode_list.append(opCode)
            else:
                push_skip_times=push_map[code]
                opCode=[loc,code,opcodes.get(code,'<------- ERROR'),byteCode[i+2:i+2+push_skip_times*2]]
                opCode_list.append(opCode)
        else:
            push_skip_times-=1
            continue
    return opCode_list

In [40]:
pprint(get_opCode_list(cont))

[['00000000x0', '60', 'PUSH1', '80'],
 ['00000000x4', '60', 'PUSH1', '40'],
 ['00000000x8', '52', 'MSTORE', ''],
 ['00000000xa', '60', 'PUSH1', '04'],
 ['00000000xe', '36', 'CALLDATASIZE', ''],
 ['0000000x10', '10', 'LT', ''],
 ['0000000x12', '61', 'PUSH2', '01c1'],
 ['0000000x18', '57', 'JUMPI', ''],
 ['0000000x1a', '63', 'PUSH4', 'ffffffff'],
 ['0000000x24',
  '7c',
  'PUSH29',
  '0100000000000000000000000000000000000000000000000000000000'],
 ['0000000x60', '60', 'PUSH1', '00'],
 ['0000000x64', '35', 'CALLDATALOAD', ''],
 ['0000000x66', '04', 'DIV', ''],
 ['0000000x68', '16', 'AND', ''],
 ['0000000x6a', '63', 'PUSH4', '02f652a3'],
 ['0000000x74', '81', 'DUP2', ''],
 ['0000000x76', '14', 'EQ', ''],
 ['0000000x78', '61', 'PUSH2', '01c6'],
 ['0000000x7e', '57', 'JUMPI', ''],
 ['0000000x80', '80', 'DUP1', ''],
 ['0000000x82', '63', 'PUSH4', '05d2035b'],
 ['0000000x8c', '14', 'EQ', ''],
 ['0000000x8e', '61', 'PUSH2', '01ee'],
 ['0000000x94', '57', 'JUMPI', ''],
 ['0000000x96', '80', 'DUP1